In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.figure_factory as ff
import json
import geojson
import requests
import plotly.graph_objs as go
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%matplotlib notebook
pd.options.display.max_columns = 150
pd.options.display.max_rows = 2000

# Insertion du dataset Pricing

In [3]:
price = pd.read_csv("../csv/prices.csv", sep=',', encoding='utf-8')
price['jour'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y-%m-%d')
price['month'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y-%m')
price['year'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y')
price = price.rename(columns={'price value':'price_value'})
price.sample(2)

,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year
65837,65837,42751,ab9164dc74d57a5d,NaN,Pompes Funèbres et Marbrerie Stopin,51.017557,2.323893,"15, rue de l’Egalité",59140,Dunkerque,328614512.0,Agence-Dunkerque-2@DIGNITE.FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-09:00-18:00|2-09:00-18:00|3-09:00-18:00|4-09...,undertaker,0.0,0.0,NaN,NaN,NaN
87423,87423,48191,368b480cddd0ced1,NaN,Cimetière de l’Est,46.342526,2.618265,46 Espl. du Souvenir Français,3100,Montluçon,470025523.0,cimetiere@mairie-montlucon.fr,NaN,Caveau provisoire 3e mois,Inhumation,temporary-cellar-3emonth,0.0,Cimetière de l’Est,2020.0,2020-11-12 11:53:43.903509,NaN,NaN,NaN,NaN,2020-11-12,2020-11,2020


# ajout des départements avec les noms dans le dataset

In [4]:
# je récupère le numéro du département à 2 chiffres (attention aux 10 premiers départements)
def get_department(postal_code):
    postal_code = str(postal_code)
    return postal_code[:2]

# Traitement des CP incomplets
price['postal_code'] = price['postal_code'].fillna('98').astype(str)
price['postal_code'] = price['postal_code'].str.zfill(5)
price['dpt_pf'] = price['postal_code'].apply(lambda x : get_department(x))

# Je rajoute la BDD des départements pour insérer les noms des départements
departements_info = pd.read_excel('../csv/departements-francais.xls')
number_to_departement = departements_info.set_index('NUMÉRO')['NOM'].to_dict()

#Ajout du nom du département dans le dataset
def convert_dpt(price):
    
    price['dpt_pf'] = price['dpt_pf'].apply(lambda x : x.lstrip('0'))
    missing_dpt_number = price[price['dpt_pf'] == ''].shape[0]
    print(f'Dropping {missing_dpt_number} out of {price.shape[0]} leads because of missing departement number')
    price = price[price['dpt_pf'] != '']
    price['dpt_pf'] = price['dpt_pf'].astype(int)
    price['departement'] = price['dpt_pf'].apply(lambda x : number_to_departement.get(x, ""))
    
    return price

price = convert_dpt(price)

# je modifie les noms des départements qui ne sont pas correctes
price['departement'] = price['departement'].str.replace("Val-D'Oise", "Val-d'Oise")
price['departement'] = price['departement'].str.replace("Seine-St-Denis", "Seine-Saint-Denis")
price['departement'] = price['departement'].str.replace("Côtes d'Armor", "Côtes-d'Armor")
price['departement'] = price['departement'].str.replace("Vandée", "Vendée")
price.sample(1)

Dropping 0 out of 95111 leads because of missing departement number


,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year,dpt_pf,departement
64497,64497,42366,5c09bd331c3e3670,NaN,Pompes Funèbres Bloyet,47.635252,-2.173118,Route de Vannes,56350,Allaire,299719455.0,pfbloyet@orange.fr,NaN,Démarches & formalités,Démarches & formalités,formalities,179.0,20211028_Allaire_Bloyet_Inhumation_Chambre,2021.0,2021-10-29 11:58:21.437088,1-00:00-00:00|2-00:00-00:00|3-00:00-00:00|4-00...,undertakerfunerarium,0.0,0.0,2021-10-29,2021-10,2021,56,Morbihan


# DATA Cimetière nationnal

In [5]:
# focus que sur les cimetières
price=price.fillna({'activities':'cimetiere'})
price_cim=price[price['activities']=='cimetiere']
print('nombre de tarifs de cimetières',price_cim.shape)
price_cim.sample(2)

nombre de tarifs de cimetières (53253, 29)


,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year,dpt_pf,departement
29223,29223,20022,dceac508cd5053d6,NaN,Cimetière Communal de Tavel,44.010795,4.706729,8 Route de la Commanderie,30126,Tavel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,30,Gard
45515,45515,31392,661f14a3a3ba5ad1,NaN,Cimetière Communal,43.240471,6.067434,45 Chemin Rural des Veys,83390,Cuers,NaN,cimetiere@cuers.fr,NaN,Case columbarium 15 ans,Colombarium,columbarium-case-15years,550.0,Cimetière Communal,2021.0,2021-11-15 07:48:46.616243,NaN,cimetiere,NaN,NaN,2021-11-15,2021-11,2021,83,Var


# choix de l'agrégat

In [6]:
# selection des champs 15 ans
price_cim_15 = price_cim[price_cim.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]

# selection des champs 30 ans
price_cim_30 = price_cim[price_cim.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]

# selection des champs 50 ans
price_cim_50 = price_cim[price_cim.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]

# Listing des départements

In [7]:
price_cim.departement.unique()

array(['Gers', 'Seine-Maritime', 'Pas-de-Calais', 'Pyrénées-Atlantiques',
       'Jura', 'Ille-et-Vilaine', 'Hérault', 'Isère', 'Gironde',
       'Lot-et-Garonne', 'Yonne', 'Indre', 'Nord', "Côte-d'Or", 'Tarn',
       'Charente', 'Haute-Marne', 'Hauts-de-Seine', 'Loire',
       'Haute-Garonne', 'Loire-Atlantique', 'Manche', 'Ariège',
       'Ardennes', 'Marne', 'Gard', 'Eure', 'Indre-et-Loire', 'Vendée',
       'Aisne', 'Haute-Saône', 'Lot', 'Bas-Rhin', 'Calvados', 'Rhône',
       'Eure-et-Loir', 'Ain', 'Sarthe', 'Charente-Maritime', 'Doubs',
       'Savoie', 'Morbihan', 'Oise', 'Meurthe-et-Moselle', 'Vosges',
       'Allier', 'Maine-et-Loire', 'Loiret', 'Dordogne', 'Corrèze',
       'Seine-et-Marne', 'Loir-et-Cher', 'Bouches-du-Rhône', 'Orne',
       'Finistère', 'Alpes-de-Haute-Provence', 'Somme', 'Cantal',
       'Saône-et-Loire', 'Cher', 'Drôme', 'Essonne', 'Landes', 'Mayenne',
       'Moselle', 'Haut-Rhin', 'Vienne', 'Haute-Vienne', 'Nièvre', 'Aube',
       'Vaucluse', 'Pyrénées-O

# data carte de france avec départements

# Var

In [8]:
# select the right department
price_cim_var=price_cim[price_cim['departement']=='Var']
print(price_cim_var.price_label.count())

# selection des champs 15 ans
price_cim_var_15 = price_cim_var[price_cim_var.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_var_15 = price_cim_var_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_var_30 = price_cim_var[price_cim_var.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_var_30 = price_cim_var_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_var_50 = price_cim_var[price_cim_var.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_var_50 = price_cim_var_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_var_15_mean=price_cim_var_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_15_mean.sample(2)

price_cim_var_30_mean=price_cim_var_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_30_mean.sample(2)

price_cim_var_50_mean=price_cim_var_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_50_mean.sample(2)

521


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-3places,concession-50years-4m2,concession-50years-4places,concession-50years-5m2,concession-50years-5places,concession-50years-6m2,concession-50years-6places
5,Fréjus,43.435938,6.733409,NaN,NaN,1305.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Hyères,43.126884,6.136720,NaN,NaN,1695.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-83-var.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes du Var')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_var_15, "cartes-inter/15y-concessions-var.html", "15")
generateVarMap(price_cim_var_30, "cartes-inter/30y-concessions-var.html", "30")
generateVarMap(price_cim_var_50, "cartes-inter/50y-concessions-var.html", "50")

# j'appelle la fonction de génération de carte à nouveau pour ajouter l'agrégat que je viens de créer

# le fichier cartes_inter/concessions-departements.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Alpes Maritime

In [10]:
# select the right department
price_cim_var=price_cim[price_cim['departement']=='Ille-et-Vilaine']
print(price_cim_var.price_label.count())

# selection des champs 15 ans
price_cim_var_15 = price_cim_var[price_cim_var.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_var_15 = price_cim_var_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_var_30 = price_cim_var[price_cim_var.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_var_30 = price_cim_var_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_var_50 = price_cim_var[price_cim_var.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_var_50 = price_cim_var_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_var_15_mean=price_cim_var_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_15_mean.sample(2)

price_cim_var_30_mean=price_cim_var_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_30_mean.sample(2)

price_cim_var_50_mean=price_cim_var_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_50_mean.sample(2)

1623


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-3places,concession-50years-4m2,concession-50years-4places,concession-50years-6m2,concession-50years-7m2
2,Baguer-Morvan,48.520824,-1.776992,NaN,NaN,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,La Gouesnière,48.601558,-1.895079,NaN,NaN,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-35-ille-et-vilaine.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_var_15, "cartes-inter/15y-concessions-Ille-et-Vilaine.html", "15")
generateVarMap(price_cim_var_30, "cartes-inter/30y-concessions-Ille-et-Vilaine.html", "30")
generateVarMap(price_cim_var_50, "cartes-inter/50y-concessions-Ille-et-Vilaine.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Indre et Loire

In [12]:
# select the right department
price_cim_dpt=price_cim[price_cim['departement']=='Indre-et-Loire']
print(price_cim_dpt.price_label.count())

# selection des champs 15 ans
price_cim_dpt_15 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_dpt_15 = price_cim_dpt_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_dpt_30 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_dpt_30 = price_cim_dpt_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_dpt_50 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_dpt_50 = price_cim_dpt_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_dpt_15_mean=price_cim_dpt_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_15_mean.sample(2)

price_cim_dpt_30_mean=price_cim_dpt_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_30_mean.sample(2)

price_cim_dpt_50_mean=price_cim_dpt_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_50_mean.sample(2)

945


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-4m2
67,Saint-Jean-Saint-Germain,47.084974,1.034166,NaN,NaN,450.0,NaN
17,Charnizay,46.914686,0.986432,NaN,NaN,100.0,NaN


In [13]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-37-indre-et-loire.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_dpt_15, "cartes-inter/15y-concessions-indre-et-loire.html", "15")
generateVarMap(price_cim_dpt_30, "cartes-inter/30y-concessions-indre-et-loire.html", "30")
generateVarMap(price_cim_dpt_50, "cartes-inter/50y-concessions-indre-et-loire.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Gironde


In [14]:
# select the right department
price_cim_dpt=price_cim[price_cim['departement']=='Gironde']
print(price_cim_dpt.price_label.count())

# selection des champs 15 ans
price_cim_dpt_15 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_dpt_15 = price_cim_dpt_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_dpt_30 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_dpt_30 = price_cim_dpt_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_dpt_50 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_dpt_50 = price_cim_dpt_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_dpt_15_mean=price_cim_dpt_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_15_mean.sample(2)

price_cim_dpt_30_mean=price_cim_dpt_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_30_mean.sample(2)

price_cim_dpt_50_mean=price_cim_dpt_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_50_mean.sample(2)

818


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-3places,concession-50years-4m2,concession-50years-4places,concession-50years-5m2,concession-50years-5places,concession-50years-6m2,concession-50years-6places,concession-50years-7m2,concession-50years-8m2
33,Sauveterre-de-Guyenne,44.717058,-0.100026,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Beychac-et-Caillau,44.879039,-0.400130,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-33-gironde.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_dpt_15, "cartes-inter/15y-concessions-gironde.html", "15")
generateVarMap(price_cim_dpt_30, "cartes-inter/30y-concessions-gironde.html", "30")
generateVarMap(price_cim_dpt_50, "cartes-inter/50y-concessions-gironde.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

In [16]:
# Haut de Seine

In [17]:
# select the right department
price_cim_dpt=price_cim[price_cim['departement']=='Hauts-de-Seine']
print(price_cim_dpt.price_label.count())

# selection des champs 15 ans
price_cim_dpt_15 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_dpt_15 = price_cim_dpt_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_dpt_30 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_dpt_30 = price_cim_dpt_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_dpt_50 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_dpt_50 = price_cim_dpt_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_dpt_15_mean=price_cim_dpt_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_15_mean.sample(2)

price_cim_dpt_30_mean=price_cim_dpt_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_30_mean.sample(2)

price_cim_dpt_50_mean=price_cim_dpt_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_50_mean.sample(2)

335


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-4m2,concession-50years-4places,concession-50years-5m2,concession-50years-6m2
10,La Garenne-Colombes,48.901485,2.232347,753.0,1800.00,NaN,NaN,NaN,NaN,NaN,NaN
14,Montrouge,48.821106,2.321081,NaN,2028.43,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-92-hauts-de-seine.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_dpt_15, "cartes-inter/15y-concessions-Hauts-de-Seine.html", "15")
generateVarMap(price_cim_dpt_30, "cartes-inter/30y-concessions-Hauts-de-Seine.html", "30")
generateVarMap(price_cim_dpt_50, "cartes-inter/50y-concessions-Hauts-de-Seine.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Rhône

In [19]:
# select the right department
price_cim_dpt=price_cim[price_cim['departement']=='Rhône']
print(price_cim_dpt.price_label.count())

# selection des champs 15 ans
price_cim_dpt_15 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_dpt_15 = price_cim_dpt_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_dpt_30 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_dpt_30 = price_cim_dpt_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_dpt_50 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_dpt_50 = price_cim_dpt_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_dpt_15_mean=price_cim_dpt_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_15_mean.sample(2)

price_cim_dpt_30_mean=price_cim_dpt_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_30_mean.sample(2)

price_cim_dpt_50_mean=price_cim_dpt_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_50_mean.sample(2)

971


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-3places,concession-50years-4m2,concession-50years-5m2,concession-50years-6places
13,Jons,45.803775,5.085507,NaN,NaN,458.0,NaN,NaN,NaN,916.0,NaN,NaN
3,Cailloux-sur-Fontaines,45.849942,4.872394,NaN,NaN,600.0,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-69-rhone.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_dpt_15, "cartes-inter/15y-concessions-Rhône.html", "15")
generateVarMap(price_cim_dpt_30, "cartes-inter/30y-concessions-Rhône.html", "30")
generateVarMap(price_cim_dpt_50, "cartes-inter/50y-concessions-Rhône.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Yonne

In [21]:
# select the right department
price_cim_dpt=price_cim[price_cim['departement']=='Yonne']
print(price_cim_dpt.price_label.count())

# selection des champs 15 ans
price_cim_dpt_15 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_dpt_15 = price_cim_dpt_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_dpt_30 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_dpt_30 = price_cim_dpt_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_dpt_50 = price_cim_dpt[price_cim_dpt.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_dpt_50 = price_cim_dpt_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_dpt_15_mean=price_cim_dpt_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_15_mean.sample(2)

price_cim_dpt_30_mean=price_cim_dpt_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_30_mean.sample(2)

price_cim_dpt_50_mean=price_cim_dpt_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_dpt_50_mean.sample(2)

55


price_label,city_name,lat,lng,concession-50years-2m2,concession-50years-3m2,concession-50years-4m2
5,Joigny,47.984558,3.398660,500.0,NaN,NaN
1,Auxerre,47.789818,3.564197,NaN,NaN,1779.0


In [22]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('../geojson/communes-89-yonne.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de l Ille et Vilaine')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_dpt_15, "cartes-inter/15y-concessions-Yonne.html", "15")
generateVarMap(price_cim_dpt_30, "cartes-inter/30y-concessions-Yonne.html", "30")
generateVarMap(price_cim_dpt_50, "cartes-inter/50y-concessions-Yonne.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus